In [2]:
from rdflib import Graph, URIRef, Literal
import json

In [3]:
g = Graph()
g.parse("coffee_data.ttl", format="ttl")

def convert_coffee_data(graph):
    coffee_lots = []
    
    # Query for all CoffeeLot instances
    query = """
    PREFIX : <http://coffee-quality.org/ontology/>
    SELECT ?lot ?species ?country ?harvestYear ?color ?processingMethod
           ?aromaScore ?flavorScore ?aftertasteScore ?acidityScore ?bodyScore 
           ?balanceScore ?uniformityScore ?cleanCupScore ?sweetnessScore 
           ?cupperPoints ?totalCupPoints ?tasteCategories ?isAromatic 
           ?isComplex ?isFullBodied ?isLingering ?isSmooth ?isSour ?isSweet
           ?qualityTier ?brewingRecommendation
    WHERE {
        ?lot a :CoffeeLot ;
            :hasSpecies ?species ;
            :fromCountry ?country ;
            :harvestYear ?harvestYear ;
            :hasColor ?color ;
            :hasProcessingMethod ?processingMethod ;
            :aromaScore ?aromaScore ;
            :flavorScore ?flavorScore ;
            :aftertasteScore ?aftertasteScore ;
            :acidityScore ?acidityScore ;
            :bodyScore ?bodyScore ;
            :balanceScore ?balanceScore ;
            :uniformityScore ?uniformityScore ;
            :cleanCupScore ?cleanCupScore ;
            :sweetnessScore ?sweetnessScore ;
            :cupperPoints ?cupperPoints ;
            :totalCupPoints ?totalCupPoints ;
            :tasteCategories ?tasteCategories ;
            :isAromatic ?isAromatic ;
            :isComplex ?isComplex ;
            :isFullBodied ?isFullBodied ;
            :isLingering ?isLingering ;
            :isSmooth ?isSmooth ;
            :isSour ?isSour ;
            :isSweet ?isSweet ;
            :hasQualityTier ?qualityTier .
        OPTIONAL { ?lot :hasBrewingRecommendation ?brewingRecommendation . }
    }
    """
    
    results = graph.query(query)
    
    # Group by coffee lot
    lots_dict = {}
    for row in results:
        lot_id = str(row[0])
        
        if lot_id not in lots_dict:
            lots_dict[lot_id] = {
                "id": lot_id,
                "species": str(row[1]),
                "country": str(row[2]).split("/")[-1] if "/" in str(row[2]) else str(row[2]),
                "harvestYear": str(row[3]),
                "color": str(row[4]).split("/")[-1] if "/" in str(row[4]) else str(row[4]),
                "processingMethod": str(row[5]).split("/")[-1] if "/" in str(row[5]) else str(row[5]),
                "scores": {
                    "aroma": float(row[6]),
                    "flavor": float(row[7]),
                    "aftertaste": float(row[8]),
                    "acidity": float(row[9]),
                    "body": float(row[10]),
                    "balance": float(row[11]),
                    "uniformity": float(row[12]),
                    "cleanCup": float(row[13]),
                    "sweetness": float(row[14]),
                    "cupperPoints": float(row[15]),
                    "totalCupPoints": float(row[16])
                },
                "tasteCategories": [cat.strip() for cat in str(row[17]).split(",")],
                "characteristics": {
                    "aromatic": bool(row[18]),
                    "complex": bool(row[19]),
                    "fullBodied": bool(row[20]),
                    "lingering": bool(row[21]),
                    "smooth": bool(row[22]),
                    "sour": bool(row[23]),
                    "sweet": bool(row[24])
                },
                "qualityTier": str(row[25]).split("/")[-1] if "/" in str(row[25]) else str(row[25]),
                "brewingRecommendations": []
            }
        
        # Add brewing recommendations (there can be multiple)
        if row[26]:
            brewing_rec = str(row[26]).split("/")[-1] if "/" in str(row[26]) else str(row[26])
            if brewing_rec not in lots_dict[lot_id]["brewingRecommendations"]:
                lots_dict[lot_id]["brewingRecommendations"].append(brewing_rec)
    
    return list(lots_dict.values())

# Convert and save
coffee_data = convert_coffee_data(g)
output = {
    "coffeeLots": coffee_data,
    "metadata": {
        "totalLots": len(coffee_data),
        "exportDate": "2024-01-01"  # You can make this dynamic
    }
}

with open("coffee_data.json", "w", encoding="utf-8") as f:
    json.dump(output, f, indent=2, ensure_ascii=False)